In [ ]:
from sklearn.linear_model import LogisticRegression

class DotaLanePredictor(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(n_inputs, n_outputs)

        
    def forward(self, x):